# WARNING THIS WILL DO REAL ORDERS!!!!!!!!!!!!!!!!!!!!!!!!!!

In [17]:
from IPython.core.display import HTML
HTML("""
<style>
.container { width:100% !important; }
</style>
""")


In [2]:
from ib_insync import *
ib = IB()
util.startLoop()
ib.connect(port=4001,clientId=1)

<IB connected to 127.0.0.1:4001 clientId=1>

### Combo Creater Format
* Input: List of Dicts with following values
    * sec_type in ['option','stock']
    * If sec_type == 'option':
        * strike
        * expiry
    * ratio
* Output: Bag Contract type for IB-Insync
[{'sec_type':'option','right':'call','strike':190,'expiry':'20240322','ratio':1},{'sec_type':'option','right':'call','strike':200,'expiry':'20240322','ratio':-1}])

In [26]:
bull_call_leg_dict_list=[{'sec_type':'option','right':'call','strike':190,'expiry':'20240322','ratio':1},
           {'sec_type':'option','right':'call','strike':200,'expiry':'20240322','ratio':-1}]
covered_call_leg_dict_list = [{'sec_type':'stock','ratio':100},
           {'sec_type':'option','right':'call','strike':200,'expiry':'20240322','ratio':-1}]

In [27]:
from typing import Tuple,Union

def get_option_contract_from_option_dict(symbol, option_dict):
    option_right = option_dict['right']  # Fixed variable name to match context
    right = 'C' if option_right == 'call' else 'P'
    strike = option_dict['strike']
    expiry = option_dict['expiry']
    # Removed ratio as it's not used in constructing the Option object
    option = Option(symbol=symbol, lastTradeDateOrContractMonth=expiry, strike=strike, right=right, exchange='SMART', currency='USD')
    if ib is not None:
        ib.qualifyContracts(option)
    return option

def get_contract_from_leg_dict(symbol, leg_dict):
    sec_type = leg_dict['sec_type']
    if sec_type == 'option':
        return get_option_contract_from_option_dict(symbol, leg_dict)  # Passed leg_dict directly
    elif sec_type == 'stock':
        stock = Stock(symbol=symbol, exchange='SMART', currency='USD')
        if ib is not None:
            ib.qualifyContracts(stock)
        return stock
    else:
        raise ValueError("sec_type must be 'stock' or 'option'")  # Raises an error if sec_type is not recognized

        
def get_combo_from_contract_ratio_list(symbol, contract_ratio_list: list[Tuple[Contract, int]]) -> Bag:
    conid_action_ratio_list: list[tuple[int, str, int]] = [(contract.conId,'BUY' if ratio>0 else 'SELL', abs(ratio)) for contract,ratio in contract_ratio_list]
    combo_legs: list[ComboLeg] = [ComboLeg(conId=conId,action=action,ratio=ratio,exchange='SMART') for conId,action,ratio in conid_action_ratio_list]
    combo: Bag = Bag(symbol=symbol,conId=-1,comboLegs=combo_legs,exchange='SMART',currency='USD')
    combo_conId = get_combo_conId(combo)
    combo.combo_conId = combo_conId
    return combo

def get_combo_conId(combo:Bag):
    combo_legs = combo.comboLegs
    return "_".join([str(leg.conId) for leg in combo_legs])

def get_combo(symbol, leg_dict_list: list[dict]):
    contract_ratio_list = [(get_contract_from_leg_dict(symbol, leg_dict), leg_dict['ratio']) for leg_dict in
                           leg_dict_list]
    return get_combo_from_contract_ratio_list(symbol, contract_ratio_list)


In [28]:
bull_call=get_combo(symbol="TSLA",leg_dict_list=bull_call_leg_dict_list)
bull_call

Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)])

In [29]:
ticker_bull_call = ib.reqMktData(bull_call,"",False,False)

In [45]:
ticker_bull_call.marketPrice()

1.685

### You can just use trade.fillEvent

In [35]:
def on_fill_event(trade:Trade,fill:Fill):
    print("ON TRADE FILL EVENT")
    print("TRADE ORDER STATUS",trade.orderStatus.status)
    print("TRADE CONTRACT",trade.contract)
    print("FILL CONTRACT",fill.contract)
    print("PERM ID",fill.execution.permId)
    print("SHARES",fill.execution.shares)
    print("SIDE",fill.execution.side)
    print("PRICE",fill.execution.price)
    print(" ")
    print(" ")
    
def on_filled_event(trade:Trade):
    print("ON TRADE FILLED EVENT")
    print("TRADE CONTRACT",trade.contract)
    print("TRADE ORDER STATUS",trade.orderStatus.status)
    print(" ")
    print(" ")
    
 
def on_execution_details(trade,fill):
    print("ON EXECUTION DETAILS EVENT")
    print("STATUS",trade.orderStatus.status)
    print("FILL CONTRACT",fill.contract)
    print("SHARES",fill.execution.shares)
    print("SIDE",fill.execution.side)
    print(" ")
    print(" ")
    

    

def place_order_with_callbacks(contract,order):
    trade=ib.placeOrder(contract,order)
    trade.fillEvent+=on_fill_event
    trade.filledEvent+=on_filled_event
    return trade
ib.execDetailsEvent.clear()
ib.execDetailsEvent+=on_execution_details

In [46]:
order = LimitOrder("BUY",1,1.71)
trade = place_order_with_callbacks(bull_call,order)

ON EXECUTION DETAILS EVENT
STATUS Submitted
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
SHARES 1.0
SIDE BOT
 
 
ON EXECUTION DETAILS EVENT
STATUS Filled
FILL CONTRACT Option(conId=681737259, symbol='TSLA', lastTradeDateOrContractMonth='20240322', strike=200.0, right='C', multiplier='100', exchange='PSE', currency='USD', localSymbol='TSLA  240322C00200000', tradingClass='TSLA')
SHARES 1.0
SIDE SLD
 
 
ON EXECUTION DETAILS EVENT
STATUS Filled
FILL CONTRACT Option(conId=681737225, symbol='TSLA', lastTradeDateOrContractMonth='20240322', strike=190.0, right='C', multiplier='100', exchange='PSE', currency='USD', localSymbol='TSLA  240322C00190000', tradingClass='TSLA')
SHARES 1.0
SIDE BOT
 
 


In [41]:
this_week_call = get_option_contract_from_option_dict('TSLA', {'sec_type':'option','right':'call','strike':190,'expiry':'20240308'})
this_week_call

Option(conId=680390304, symbol='TSLA', lastTradeDateOrContractMonth='20240308', strike=190.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='TSLA  240308C00190000', tradingClass='TSLA')

In [43]:
def on_fill_event(trade:Trade,fill:Fill):
    child_action = 'SELL' if fill.execution.side=='BOT' else 'BUY' 
    qty = fill.execution.shares
    price = fill.execution.price
    take_profit_price = price + 0.02 if child_action=='SELL' else price - 0.02
    child_limit_order = LimitOrder(action=child_action,totalQuantity=qty,lmtPrice=take_profit_price)
    if fill.contract.secType=='BAG':
        ib.placeOrder(this_week_call,child_limit_order)
        

In [44]:
def place_order_with_callbacks(contract,order):
    trade=ib.placeOrder(contract,order)
    trade.fillEvent+=on_fill_event
    return trade

In [18]:
trade.orderStatus

OrderStatus(orderId=159, status='PreSubmitted', filled=0.0, remaining=1.0, avgFillPrice=0.0, permId=1108837094, parentId=0, lastFillPrice=0.0, clientId=1, whyHeld='', mktCapPrice=0.0)

Value (Trade(contract=Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=LimitOrder(orderId=159, clientId=1, permId=1108837094, action='BUY', totalQuantity=1.0, lmtPrice=1.63, auxPrice=0.0), orderStatus=OrderStatus(orderId=159, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=1.63, permId=1108837094, parentId=0, lastFillPrice=1.63, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB'), execution=Execution(execId='000318a5.65e7feed.01.01', time=datetime.datetime(2024, 3, 6, 16, 21, 41, tzinfo=datetime.timezone.utc), acctNumber='U8765400', exchange='SMAR

ON EXECUTION DETAILS FILEVENT
STATUS Submitted
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
SHARES 1.0
SIDE BOT
ON TRADE FILL EVENT
TRADE ORDER STATUS Submitted
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)])
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
PERM ID 1108837094
SHARES 1.0
SIDE BOT
PRICE 1.63
ON TRADE FILLED EVENT
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleS

Value (Trade(contract=Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=LimitOrder(orderId=158, clientId=1, permId=1108837093, action='BUY', totalQuantity=1.0, lmtPrice=1.62, auxPrice=0.0), orderStatus=OrderStatus(orderId=158, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=1.62, permId=1108837093, parentId=0, lastFillPrice=1.62, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB'), execution=Execution(execId='000318a5.65e7feef.01.01', time=datetime.datetime(2024, 3, 6, 16, 21, 46, tzinfo=datetime.timezone.utc), acctNumber='U8765400', exchange='SMAR

ON EXECUTION DETAILS FILEVENT
STATUS Submitted
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
SHARES 1.0
SIDE BOT
ON TRADE FILL EVENT
TRADE ORDER STATUS Submitted
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)])
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
PERM ID 1108837093
SHARES 1.0
SIDE BOT
PRICE 1.62
ON TRADE FILLED EVENT
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleS

Value (Trade(contract=Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=LimitOrder(orderId=157, clientId=1, permId=1108837092, action='BUY', totalQuantity=1.0, lmtPrice=1.61, auxPrice=0.0), orderStatus=OrderStatus(orderId=157, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=1.61, permId=1108837092, parentId=0, lastFillPrice=1.61, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB'), execution=Execution(execId='000318a5.65e7fef1.01.01', time=datetime.datetime(2024, 3, 6, 16, 22, 31, tzinfo=datetime.timezone.utc), acctNumber='U8765400', exchange='SMAR

ON EXECUTION DETAILS FILEVENT
STATUS Submitted
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
SHARES 1.0
SIDE BOT
ON TRADE FILL EVENT
TRADE ORDER STATUS Submitted
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)])
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
PERM ID 1108837092
SHARES 1.0
SIDE BOT
PRICE 1.61
ON TRADE FILLED EVENT
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleS

Value (Trade(contract=Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)]), order=LimitOrder(orderId=156, clientId=1, permId=1108837091, action='BUY', totalQuantity=1.0, lmtPrice=1.59, auxPrice=0.0), orderStatus=OrderStatus(orderId=156, status='Filled', filled=1.0, remaining=0.0, avgFillPrice=1.59, permId=1108837091, parentId=0, lastFillPrice=1.59, clientId=1, whyHeld='', mktCapPrice=0.0), fills=[Fill(contract=Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB'), execution=Execution(execId='000318a5.65e7fef3.01.01', time=datetime.datetime(2024, 3, 6, 16, 25, 47, tzinfo=datetime.timezone.utc), acctNumber='U8765400', exchange='SMAR

ON EXECUTION DETAILS FILEVENT
STATUS Submitted
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
SHARES 1.0
SIDE BOT
ON TRADE FILL EVENT
TRADE ORDER STATUS Submitted
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1), ComboLeg(conId=681737259, ratio=1, action='SELL', exchange='SMART', openClose=0, shortSaleSlot=0, designatedLocation='', exemptCode=-1)])
FILL CONTRACT Bag(conId=28812380, symbol='TSLA', right='C', exchange='SMART', currency='USD', localSymbol='TSLA', tradingClass='COMB')
PERM ID 1108837091
SHARES 1.0
SIDE BOT
PRICE 1.59
ON TRADE FILLED EVENT
TRADE CONTRACT Bag(conId=-1, symbol='TSLA', exchange='SMART', currency='USD', comboLegs=[ComboLeg(conId=681737225, ratio=1, action='BUY', exchange='SMART', openClose=0, shortSaleS

In [10]:
3

3

In [ ]:
changed_strike_call = 
order = LimitOrder("BUY",1,1.59)
trade = place_order_with_callbacks(bull_call,order)